# To do
- 各個人で外勤日が決まっているので、特定の曜日を予め決めておくと、その曜日は計算しない、あるいは研究日に自動でなるようにする

In [1]:
import pandas as pd
import datetime
from datetime import timedelta
import numpy as np

#下記のコードでutils.pyから関数を呼び出す
from utils import start, end, make_cal

In [2]:
#基準時刻を設定
start_criteria = '09:00'
start_criteria = datetime.datetime.strptime(start_criteria, '%H:%M').time()

end_criteria = '17:00'
end_criteria = datetime.datetime.strptime(end_criteria, '%H:%M').time()

end_criteria_saturday = '13:00'
end_criteria_saturday = datetime.datetime.strptime(end_criteria_saturday, '%H:%M').time()

# 設定ファイルの読み込み
df = pd.read_excel("202201_shift.xlsx", dtype='object', engine="openpyxl")

#ある行がすべて欠損している場合、その行を削除
df.dropna(how='all', inplace = True)

#読み込んだファイルから年、月の取得
year, month = df.iloc[0,1:][0], df.iloc[0,1:][3]

#始業時刻
shigyou = df.iloc[4,5:].fillna(start_criteria) #以前はlist5で定義
#終業時刻
syugyou = df.iloc[5,5:].fillna(end_criteria) #以前はlist6で定義

#カレンダー作成
cal_ext = make_cal(year, month)

df_day = pd.DataFrame(np.arange(1,32)).set_axis(['日付'], axis='columns').astype(str)
df_sigyo = pd.DataFrame(shigyou).reset_index(drop=True).set_axis(['始業時刻'], axis='columns')
df_syugyo = pd.DataFrame(syugyou).reset_index(drop=True).set_axis(['終業時刻'], axis='columns')

df_work=pd.concat([df_day,df_sigyo,df_syugyo], axis=1)
df_work=pd.merge(cal_ext,df_work, on ="日付")

df_work['始業基準時刻']=start_criteria
df_work['終業基準時刻']=end_criteria
for i in range(len(df_work)):
    if df_work.iloc[i,2]=="土曜":
        df_work.iloc[i,6]=end_criteria_saturday
df_work

#早朝勤務分の時間外労働
diff_shigyo_start = [] #以前はlist7で定義
for i in range(len(df_work)):
    diff_shigyo_start.append(start(df_work.iloc[i,3], df_work.iloc[i,5]))
    
#残業分の時間外労働    
diff_syugyo_end = [] #以前はlist8で定義
for i in range(len(df_work)):
    if end(df_work.iloc[i,4], df_work.iloc[i,6])>=0:
        test2=end(df_work.iloc[i,4], df_work.iloc[i,6])
    else:
        #終業時間が24時を超えるとき x時-17時で＋24時間すると残業時間になる
        #（1時→－16時間；残業は8時間）
        test2 = end(df_work.iloc[i,4], df_work.iloc[i,6])+24*60
    diff_syugyo_end.append(test2)
    
df_work["早朝勤務"]=diff_shigyo_start
df_work["残業勤務"]=diff_syugyo_end
df_work["合計残業"]=df_work["早朝勤務"]+df_work["残業勤務"]
df_work

C:\Users\hs691\anaconda3\envs\torch_cv\lib\site-packages\openpyxl\worksheet\_reader.py:300: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


,日付,曜日,扱い,始業時刻,終業時刻,始業基準時刻,終業基準時刻,早朝勤務,残業勤務,合計残業
0,1,土,土曜,07:00:00,18:00:00,09:00:00,13:00:00,120.0,300.0,420.0
1,2,日,休日,07:00:00,19:00:00,09:00:00,17:00:00,120.0,120.0,240.0
2,3,月,平日,09:00:00,01:00:00,09:00:00,17:00:00,0.0,480.0,480.0
3,4,火,平日,09:00:00,17:00:00,09:00:00,17:00:00,0.0,0.0,0.0
4,5,水,平日,09:00:00,17:00:00,09:00:00,17:00:00,0.0,0.0,0.0
5,6,木,平日,09:00:00,17:00:00,09:00:00,17:00:00,0.0,0.0,0.0
6,7,金,平日,09:00:00,17:00:00,09:00:00,17:00:00,0.0,0.0,0.0
7,8,土,土曜,09:00:00,17:00:00,09:00:00,13:00:00,0.0,240.0,240.0
8,9,日,休日,09:00:00,17:00:00,09:00:00,17:00:00,0.0,0.0,0.0
9,10,月,平日,09:00:00,17:00:00,09:00:00,17:00:00,0.0,0.0,0.0


# 大きく修正したコード

In [ ]:
#for文を使わないでnumpyを使って処理
df_day = pd.DataFrame(np.arange(1,32)).set_axis(['日付'], axis='columns').astype(str)

"""
#日付表の作成
list_day=[]
for i in range(31):
    list_day.append(i+1)
print(type(list_day[2]))

df_day=pd.DataFrame(data=list_day,columns=["日付"])
df_day=df_day.astype(str)
df_day
"""

In [36]:
#リスト変換しなくても直接pd.DataFrame(shigyou)と書けば良い
df_sigyo = pd.DataFrame(shigyou).reset_index(drop=True).set_axis(['始業時刻'], axis='columns')

"""
list_sigyo=[]
list_sigyo.append(list5)
list_sigyo
df_sigyo = pd.DataFrame(list_sigyo)
df_sigyo2=df_sigyo.T
df_sigyo2.columns=["始業時刻"]
df_sigyo3=df_sigyo2.reset_index(drop=True)
df_day=df_day.astype(str)
df_sigyo3
"""

In [ ]:
#リスト変換しなくても直接pd.DataFrame(shigyou)と書けば良い
df_syugyo = pd.DataFrame(syugyou).reset_index(drop=True).set_axis(['終業時刻'], axis='columns')

"""
list_syugyo=[]
list_syugyo.append(list6)
list_syugyo
df_syugyo = pd.DataFrame(list_syugyo)
df_syugyo2=df_syugyo.T
df_syugyo2.columns=["終業時刻"]
df_syugyo3=df_syugyo2.reset_index(drop=True) 
"""

In [ ]:
#元のstart関数のままだと土曜日の終業時間13：00がうまく処理できない。
#このためstar,end関数は作成したカレンダーから2つの引数(始業時間：start(df_work.iloc[i,3], 始業基準時刻：df_work.iloc[i,5]))を取って
#差分の時間を計算する関数に修正した。

#この関数はutils.pyに記載しています。
def start(sigyo, start_cri_t):
    #関数startの引数start_timeを定義
    #一時変数dt_sigyoutime
    #returnを使って始業時刻からの残業代を計算
    """
    Args:
        datetime.time #df_workから引数を取る
    Returns:
        float: 分
    """
    dt_sigyotime = datetime.datetime.combine(datetime.date.today(), sigyo)
    dt_start_cri_t = datetime.datetime.combine(datetime.date.today(), start_cri_t)
    return (dt_start_cri_t - dt_sigyotime).total_seconds()/60

"""
def start(start_time):
    dt_sigyotime= datetime.datetime.combine(datetime.date.today(), start_time)
    return (dt2-dt_sigyotime).total_seconds()/60
"""